<a href="https://colab.research.google.com/github/aksesss/PIS/blob/master/labs/lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install tensorflow-gpu==1.15.0

In [2]:
import numpy as np
import tensorflow as tf
from collections import Counter
from sklearn.datasets import fetch_20newsgroups
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
categories = ['comp.os.ms-windows.misc', 'talk.politics.misc', 'talk.politics.mideast']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

def text2vocab(data_train, data_test):
  vocab = Counter()
  for text in data_train.data:
    for word in text.split(' '):
      vocab[word.lower()] += 1
  for text in data_test.data:
    for word in text.split(' '):
      vocab[word.lower()] += 1
  return vocab

def get_word_2_index(vocab):
  word2index = {}
  for i, word in enumerate(vocab):
    word2index[word.lower()] = i
  return word2index

vocab = text2vocab(newsgroups_train, newsgroups_test)
total_words = len(vocab)
word2index = get_word_2_index(vocab)


In [0]:
def get_batch(df, i, batch_size):
  batches = []
  results = []
  texts = df.data[i * batch_size:i * batch_size + batch_size]
  categories = df.target[i * batch_size:i * batch_size + batch_size]
  for text in texts:
    layer = np.zeros(total_words, dtype=float)
    for word in text.split(' '):
      layer[word2index[word.lower()]] += 1
    batches.append(layer)

  for category in categories:
    y = np.zeros((4), dtype=float)
    if category == 0:
      y[0] = 1.
    elif category == 1:
      y[1] = 1.
    elif category == 2:
      y[2] = 1.
    else:
      y[3] = 1.
  results.append(y)
  
  return np.array(batches), np.array(results)

In [0]:
# Параметры обучения
learning_rate = 0.01
training_epochs = 10
batch_size = 150
display_step = 1

# Network Parameters
n_hidden_1 = 2# скрытый слой
n_hidden_2 = 10 # скрытый слой
n_hidden_3 = 2 # скрытый слой

n_input = total_words # количество уникальных слов в наших текстах
n_classes = 4 # 4 класса

input_tensor = tf.placeholder(tf.float32,[None, n_input],name="input")
output_tensor = tf.placeholder(tf.float32,[None, n_classes],name="output")


In [6]:
def multilayer_perceptron(input_tensor, weights, biases):
  # скрытый слой
  layer_1_multiplication = tf.matmul(input_tensor, weights['h1'])
  layer_1_addition = tf.add(layer_1_multiplication, biases['b1'])
  layer_1 = tf.nn.relu(layer_1_addition)

  # скрытый слой
  layer_2_multiplication = tf.matmul(layer_1, weights['h2'])
  layer_2_addition = tf.add(layer_2_multiplication, biases['b2'])
  layer_2 = tf.nn.relu(layer_2_addition)
  
  # скрытый слой
  layer_3_multiplication = tf.matmul(layer_2, weights['h3'])
  layer_3_addition = tf.add(layer_3_multiplication, biases['b3'])
  layer_3 = tf.nn.relu(layer_3_addition)

  # выходной слой
  out_layer_multiplication = tf.matmul(layer_3, weights['out'])
  out_layer_addition = out_layer_multiplication + biases['out']
  
  return out_layer_addition

# инициализация параметров сети
weights = {
  'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
  'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
  'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
  'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
  'b1': tf.Variable(tf.random_normal([n_hidden_1])),
  'b2': tf.Variable(tf.random_normal([n_hidden_2])),
  'b3': tf.Variable(tf.random_normal([n_hidden_3])),
  'out': tf.Variable(tf.random_normal([n_classes]))
}
# создание модели
prediction = multilayer_perceptron(input_tensor, weights, biases)

# Фукнция потерь
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=output_tensor))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

init = tf.global_variables_initializer()



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
import time
# Запуск
with tf.Session() as sess:
  sess.run(init)
  start_time = time.time()

  total_test_data = len(newsgroups_test.target)
  total_train_data = len(newsgroups_train.target)

  full_x_test,full_y_test = get_batch(newsgroups_test,0,total_test_data)
  full_x_train,full_y_train = get_batch(newsgroups_train,0,total_train_data)

  # Цикл обучения
  for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(len(newsgroups_train.data)/batch_size)
    # Проход по всем батчам
    for i in range(total_batch):
      batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
      c,_ = sess.run([loss,optimizer], feed_dict={input_tensor: batch_x,output_tensor:batch_y})
      # Вычисляем среднее фукнции потерь
      avg_cost += c / total_batch
    print("Эпоха:", '%04d' % (epoch+1), 'Время:%07d'%(time.time() - start_time), "loss=", "{:.16f}".format(avg_cost))

    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(output_tensor, 1))
    # Расчет точности
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
    
    #print("Точность треин:", accuracy.eval({input_tensor: batch_x, output_tensor: batch_y}))
    print("\tТочность тест :", accuracy.eval({input_tensor: full_x_test, output_tensor: full_y_test}))
    print("\tТочность треин:", accuracy.eval({input_tensor: full_x_train, output_tensor: full_y_test}))

  print("Обучение завершено!")

# Тестирование
  correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(output_tensor, 1))
  # Расчет точности
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  total_test_data = len(newsgroups_test.target)
  batch_x_test,batch_y_test = get_batch(newsgroups_test,0,total_test_data)
  #print("Точность треин:", accuracy.eval({input_tensor: batch_x, output_tensor: batch_y}))
  print("Точность тест :", accuracy.eval({input_tensor: batch_x_test, output_tensor: batch_y_test}))

Эпоха: 0001 Время:0000013 loss= 2.2812974810600282
	Точность тест : 0.9685185
	Точность треин: 0.9512346
Эпоха: 0002 Время:0000017 loss= 1.9397820472717284
	Точность тест : 0.97962964
	Точность треин: 0.9617284
Эпоха: 0003 Время:0000021 loss= 1.8329634666442873
	Точность тест : 0.9787037
	Точность треин: 0.9537037
Эпоха: 0004 Время:0000025 loss= 1.7523664414882660
	Точность тест : 0.9759259
	Точность треин: 0.95308644
Эпоха: 0005 Время:0000029 loss= 1.6814268022775647
	Точность тест : 0.9759259
	Точность треин: 0.95
Эпоха: 0006 Время:0000033 loss= 1.6166168749332428
	Точность тест : 0.97407407
	Точность треин: 0.94876546
Эпоха: 0007 Время:0000037 loss= 1.5579245269298552
	Точность тест : 0.97407407
	Точность треин: 0.9506173
Эпоха: 0008 Время:0000042 loss= 1.5041313827037812
	Точность тест : 0.9611111
	Точность треин: 0.94135803
Эпоха: 0009 Время:0000047 loss= 1.4561185896396638
	Точность тест : 0.9638889
	Точность треин: 0.9419753
Эпоха: 0010 Время:0000052 loss= 1.4111862778663633
	То